In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.image as img
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam,rmsprop,Adadelta
from keras.metrics import categorical_crossentropy
from batch_generator import batch_generator
from metrics import (dice_coef,dice_loss)
from models import create
from model_2pathed import create_2pathed
from model_2pathed_shallow import create_2pathed_shallow
from scheduler import scheduler
from image_generator import generate_image
from glob import glob
import os


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
samples_paths=sorted(glob("./Data/Train/samples_processed/*.npy"))
masks_paths=sorted(glob("./Data/Train/masks_reformatted/*.npy"))


In [3]:
samples_paths

['./Data/Train/samples_processed\\sample_001.npy',
 './Data/Train/samples_processed\\sample_002.npy',
 './Data/Train/samples_processed\\sample_003.npy',
 './Data/Train/samples_processed\\sample_004.npy',
 './Data/Train/samples_processed\\sample_005.npy',
 './Data/Train/samples_processed\\sample_006.npy',
 './Data/Train/samples_processed\\sample_007.npy',
 './Data/Train/samples_processed\\sample_008.npy',
 './Data/Train/samples_processed\\sample_009.npy',
 './Data/Train/samples_processed\\sample_010.npy',
 './Data/Train/samples_processed\\sample_011.npy',
 './Data/Train/samples_processed\\sample_012.npy',
 './Data/Train/samples_processed\\sample_013.npy',
 './Data/Train/samples_processed\\sample_014.npy',
 './Data/Train/samples_processed\\sample_015.npy',
 './Data/Train/samples_processed\\sample_016.npy',
 './Data/Train/samples_processed\\sample_017.npy',
 './Data/Train/samples_processed\\sample_018.npy',
 './Data/Train/samples_processed\\sample_019.npy',
 './Data/Train/samples_processe

In [4]:
train_sample_paths,test_sample_paths=train_test_split(samples_paths,test_size=0.25)

In [6]:
len(train_sample_paths)

276

In [5]:
traingen=batch_generator(train_sample_paths,16) #Mini batch size of 16
testgen=batch_generator(test_sample_paths,16)


In [2]:
model=create_2pathed_shallow()

In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                28, 3)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 126, 126, 12  656         ['input_1[0][0]']                
                                6, 8)                                                             
                                                                                                  
 conv3d_6 (Conv3D)              (None, 126, 126, 12  656         ['input_1[0][0]']                
                                6, 8)                                                         

In [11]:
sample,mask=next(traingen)

In [12]:
sample.shape

(16, 128, 128, 128, 3)

In [13]:
mask.shape

(16, 128, 128, 128, 4)

In [7]:
#Compute steps per epoch
def steps(m,batchsize):
    return (m+batchsize-1)//batchsize

In [8]:
train_steps=steps(len(train_sample_paths),16)
test_steps=steps(len(test_sample_paths),16)


In [16]:
train_steps

18

In [10]:
import datetime

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt,loss=dice_loss,metrics=[dice_coef])

#callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
schedule=tf.keras.callbacks.LearningRateScheduler(scheduler)
nanterminate=tf.keras.callbacks.TerminateOnNaN()
earlystop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.05)
checkpoints=tf.keras.callbacks.ModelCheckpoint(filepath="./model_checkpoints/2pathed_shallow",save_weights_only=False,save_freq='epoch',save_best_only=True,monitor='dice_coef',mode='max')


In [12]:
history=model.fit(traingen,steps_per_epoch=train_steps,epochs=50,validation_data=testgen,validation_steps=train_steps,callbacks=[tensorboard_callback,schedule,nanterminate,earlystop,checkpoints])

Epoch 1/50
